In [16]:
import pandas as pd
from sqlalchemy import create_engine
import cred_pg as c
import json
from collections import OrderedDict



engine = create_engine(
    f'postgresql://{c.pg_userid}:{c.pg_password}@{c.pg_host}/{c.pg_db}', 
    connect_args = {
        'options': '-c search_path=${user},ugeobln,ugm,uinsta,umisc,umobility,usozmed,public', 
        'keepalives_idle': 120
    },
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

from graphql.parser import GraphQLParser
parser = GraphQLParser()

with engine.connect() as con:
    sql = f"""
        SELECT
            conname AS constraint_name,
            n.nspname AS schema_name,
            cl.relname AS table_name,
            a.attname AS column_name,
            nt.nspname AS primary_schema,
            clt.relname AS primary_table,
            af.attname AS primary_column
        FROM pg_constraint AS c
        JOIN pg_namespace AS n ON n.oid = c.connamespace
        JOIN pg_class AS cl ON cl.oid = c.conrelid
        JOIN pg_attribute AS a ON a.attnum = ANY(c.conkey) AND a.attrelid = c.conrelid AND a.attisdropped = false
        JOIN pg_class AS clt ON clt.oid = c.confrelid
        JOIN pg_namespace AS nt ON nt.oid = clt.relnamespace
        JOIN pg_attribute AS af ON af.attnum = ANY(c.confkey) AND af.attrelid = c.confrelid AND af.attisdropped = false
        WHERE n.nspname = 'ugm' AND c.contype = 'f';
        """
    
    df_relationships = pd.read_sql_query(sql, con)

      

unique_tables_set = set(df_relationships['table_name']).union(set(df_relationships['primary_table']))

table_names = list(unique_tables_set)



relationships = []

for table in table_names:
    # Filter rows where the current table is either the source or the target
    source_rows = df_relationships[df_relationships['table_name'] == table]
    target_rows = df_relationships[df_relationships['primary_table'] == table]

    # Extract relevant information
    related_tables = list(source_rows['primary_table']) + list(target_rows['table_name'])
    related_keys = list(source_rows['column_name']) + list(target_rows['primary_column'])

    # Create a dictionary for the current table
    dict_table = {
        table: {
            'table': related_tables,
            'key': related_keys
        }
    }

    relationships.append(dict_table)


relationships


def get_field(field):
  return field.name

def get_selection(selections):
  return [s.name for s in selections]

    
def get_conditions(ast):
    where_clauses = []

    # Function to recursively traverse the AST
    def traverse(node):
        # Extract arguments from the current node
        for a in node.arguments:
            where_clauses.append([a.name, a.value])

        # Recursively apply to nested selections
        for selection in node.selections:
            traverse(selection)

    # Start traversal from the root AST node
    traverse(ast)

    return where_clauses


sql = f"""
    select *
    from INFORMATION_SCHEMA.COLUMNS
    where table_schema = 'ugm'and TABLE_NAME='kunde'
    """
with engine.connect() as con:
    table_df = pd.read_sql_query(sql, con)


### problem bei  get_selections ist gelöust 
def get_selections(ast):
    selections = []

    for field in ast.selections:
        # Add the first field name of each top-level selection
        selections.append(field.name)

        # For each selection that has nested fields, add the first field name of the nested selection
        if field.selections:
            selections.append(field.selections[0].name)

    return selections


def get_joins(ast, parent_table=None):
    def get_table_name(full_table_name):
        # Removes the schema part if it exists
        return full_table_name.split('.')[-1]

    table = get_table_name(get_field(ast))
    if parent_table is None:
        join_tables = [table]
    else:
        join_tables = [get_table_name(parent_table)]

    if ast.selections:
        for field in ast.selections:
            if field.selections:  # Field has nested selections
                nested_table = get_table_name(get_field(field))
                join_tables += get_joins(field, nested_table)
            else:
                join_tables.append(table)

    return list(OrderedDict.fromkeys(join_tables))  # Remove duplicates while preserving order






def get_where(ast):
    condition = get_conditions(ast)
    join_tables = get_joins(ast)

    # Check if there are enough join tables for the where clause construction
    if len(join_tables) < 2:
        raise ValueError("Not enough join tables to construct where clauses")

    var = "'"
    str_where = [f'{join_tables[1]}.{l} = {var}{r}{var}' for (l, r) in condition]
    return str_where



def get_join_tables_and_keys(ast):
    join_tables = get_joins(ast)
    join_keys = []

    # Initialize the loop with the first table
    i = 0
    while i < len(join_tables) - 1:
        table_name = join_tables[i]
        primary_table = join_tables[i + 1]

        # Attempt to find a direct relationship or an intermediate table
        found_key = False
        for relation in relationships:
            if table_name in relation:
                relationship_info = relation[table_name]

                if primary_table in relationship_info['table']:
                    key_index = relationship_info['table'].index(primary_table)
                    join_keys.append(relationship_info['key'][key_index])
                    found_key = True
                    break
                else:
                    for j, intermediate_table in enumerate(relationship_info['table']):
                        # Check if the intermediate table links to the target table
                        for next_relation in relationships:
                            if intermediate_table in next_relation:
                                next_relationship_info = next_relation[intermediate_table]
                                if primary_table in next_relationship_info['table']:
                                    # Add keys for both steps of the join
                                    join_keys.append(relationship_info['key'][j])
                                    next_key_index = next_relationship_info['table'].index(primary_table)
                                    join_keys.append(next_relationship_info['key'][next_key_index])

                                    # Insert the intermediate table into the join path
                                    join_tables.insert(i + 1, intermediate_table)
                                    found_key = True
                                    break
                        if found_key:
                            break
            if found_key:
                break

        # Move to the next pair of tables
        i += 1

    return join_tables, join_keys





def get_join_tables_recursive(ast, index=0, join_keys=[]):
    join_tables = get_joins(ast)

    # Base case
    if len(join_tables) - len(join_keys) == 1 or index >= len(join_tables) - 1:
        return join_tables

    tableName = join_tables[index]
    primaryTable = join_tables[index + 1]

    # Process the join tables
    for result in relationships:
        if result.get(tableName):
            temp1 = result.get(tableName)

            for t1 in temp1['table']:
                if t1 != join_tables[index - 1]:
                    for result in relationships:
                        if result.get(t1):
                            temp2 = result.get(t1)
                            for t2 in temp2['table']:
                                if t2 == primaryTable:
                                    join_tables.insert(index + 1, t1)
                                    join_tables = list(OrderedDict.fromkeys(join_tables))
                                    break

    # Recursive call for the next pair
    return get_join_tables_recursive(ast, index + 1, join_keys)
    


def get_str_joins(ast):
    join_tables, join_keys = get_join_tables_and_keys(ast)
    index = 0
    str_joins = ''
    while index < len(join_tables)-1:
        str_join = f'inner join {join_tables[index+1]} on {join_tables[index]}.{join_keys[index]} = {join_tables[index+1]}.{join_keys[index]} \n'
        index +=1
        str_joins+=str_join

    return str_joins 


def graph_to_sql(ast):
    """
    Generates a SQL query from the given AST.

    Parameters:
    ast (AST): The abstract syntax tree representing the GraphQL query.

    Returns:
    str: A SQL query string.
    """

    def create_where(conditions):
        """ Create the WHERE clause of SQL query """
        return ' AND '.join([f"{col} = '{val}'" for col, val in conditions])

    def validate_columns(columns):
        """ Validate if columns exist in the table """
        return all(col in table_df['column_name'].values for col in columns)

    table = get_field(ast)
    columns = get_selection(ast.selections)
    condition = get_conditions(ast)
    str_joins = get_str_joins(ast)

    if validate_columns(columns):
        where_clause = create_where(condition)
        sql = f"""
            SELECT jsonb_pretty(jsonb_agg(tb)::jsonb) as to_json 
            FROM (
                SELECT {', '.join(columns)} 
                FROM ugm.{table} 
                WHERE {where_clause}
            ) tb
        """
    else:
        selections = get_selections(ast)
        str_where = get_where(ast)
        sql = f"""
        SELECT jsonb_pretty(jsonb_agg(tb)::jsonb) as to_json 
        FROM (
            SELECT {', '.join(selections)} 
            FROM ugm.{table}
            {str_joins if str_joins else ''} 
            WHERE {' AND '.join(str_where)}
        ) tb
    """

    return sql


def graphql_eval(ast):

    result_json = []

    query = graph_to_sql(ast)

    with engine.connect() as con:
        df = pd.read_sql_query(query, con)

    try:

        result_json = df.iloc[0]['to_json']

        # Print the result within the function
        print(result_json)

        return None

    except Exception as e:
        print(f"Error: {e}")
        return None

    finally:
        # Close the database connection
        if engine:
            engine.dispose()
   
 
 


In [17]:
ast1 = parser.parse("""
query {
  kunde (kid: "10001"){
    name
    titel
  }
}
""")
q1 = ast1.definitions[0].selections[0]


ast2 = parser.parse("""
query {
  kunde(kid: "10001") {
    name
    bestellung {
      bestelldatum
      gesamtpreis
    }
  }
}
""")
q2 = ast2.definitions[0].selections[0]

ast3 = parser.parse("""
query {
  kunde(kid: "10001") {
    name
    bestellung (bestelldatum: "2019-01-31") {
      bestelldatum
      gesamtpreis
    }
  }
}
""")
q3 = ast3.definitions[0].selections[0]
#print(q3)

ast4 = parser.parse("""
query {
  kunde(kid: "10001") {
    name
    bestellung {
      produkt {
          bez
          laenge
      }
      bestelldatum
      gesamtpreis
    }
  }
}
""")
q4 = ast4.definitions[0].selections[0]
print(q4)

<Field: name=kunde, arguments=[<Argument: name=kid, value=10001>], selections=[<Field: name=name>, <Field: name=bestellung, selections=[<Field: name=produkt, selections=[<Field: name=bez>, <Field: name=laenge>]>, <Field: name=bestelldatum>, <Field: name=gesamtpreis>]>]>


In [18]:
print(graph_to_sql(q1))
print(graph_to_sql(q2))
print(graph_to_sql(q3))
print(graph_to_sql(q4))


            SELECT jsonb_pretty(jsonb_agg(tb)::jsonb) as to_json 
            FROM (
                SELECT name, titel 
                FROM ugm.kunde 
                WHERE kid = '10001'
            ) tb
        

        SELECT jsonb_pretty(jsonb_agg(tb)::jsonb) as to_json 
        FROM (
            SELECT name, bestellung, bestelldatum 
            FROM ugm.kunde
            inner join bestellung on kunde.kid = bestellung.kid 
 
            WHERE bestellung.kid = '10001'
        ) tb
    

        SELECT jsonb_pretty(jsonb_agg(tb)::jsonb) as to_json 
        FROM (
            SELECT name, bestellung, bestelldatum 
            FROM ugm.kunde
            inner join bestellung on kunde.kid = bestellung.kid 
 
            WHERE bestellung.kid = '10001' AND bestellung.bestelldatum = '2019-01-31'
        ) tb
    

        SELECT jsonb_pretty(jsonb_agg(tb)::jsonb) as to_json 
        FROM (
            SELECT name, bestellung, produkt 
            FROM ugm.kunde
            inner join

In [19]:
graphql_eval(q1)

[
    {
        "name": "Eck",
        "titel": "Dr."
    }
]


In [11]:
graphql_eval(q2)

[
    {
        "name": "Eck",
        "bestellung": {
            "bid": 107103,
            "kid": 10001,
            "gesamtpreis": 725.25,
            "bestelldatum": "2018-06-27"
        },
        "bestelldatum": "2018-06-27"
    },
    {
        "name": "Eck",
        "bestellung": {
            "bid": 111590,
            "kid": 10001,
            "gesamtpreis": 145.99,
            "bestelldatum": "2019-01-31"
        },
        "bestelldatum": "2019-01-31"
    },
    {
        "name": "Eck",
        "bestellung": {
            "bid": 113135,
            "kid": 10001,
            "gesamtpreis": 28.00,
            "bestelldatum": "2019-04-17"
        },
        "bestelldatum": "2019-04-17"
    },
    {
        "name": "Eck",
        "bestellung": {
            "bid": 115316,
            "kid": 10001,
            "gesamtpreis": 871.66,
            "bestelldatum": "2019-06-28"
        },
        "bestelldatum": "2019-06-28"
    },
    {
        "name": "Eck",
        "bestellung": 

In [20]:
graphql_eval(q3)

[
    {
        "name": "Eck",
        "bestellung": {
            "bid": 111590,
            "kid": 10001,
            "gesamtpreis": 145.99,
            "bestelldatum": "2019-01-31"
        },
        "bestelldatum": "2019-01-31"
    }
]


In [21]:
graphql_eval(q4)

[
    {
        "name": "Eck",
        "produkt": {
            "bez": "Lucca-Esstisch",
            "pid": 61104,
            "breite": null,
            "laenge": null
        },
        "bestellung": {
            "bid": 107103,
            "kid": 10001,
            "gesamtpreis": 725.25,
            "bestelldatum": "2018-06-27"
        }
    },
    {
        "name": "Eck",
        "produkt": {
            "bez": "Stehtisch rund",
            "pid": 61201,
            "breite": null,
            "laenge": null
        },
        "bestellung": {
            "bid": 107103,
            "kid": 10001,
            "gesamtpreis": 725.25,
            "bestelldatum": "2018-06-27"
        }
    },
    {
        "name": "Eck",
        "produkt": {
            "bez": "Gartenbank Sylt",
            "pid": 62102,
            "breite": null,
            "laenge": 150
        },
        "bestellung": {
            "bid": 107103,
            "kid": 10001,
            "gesamtpreis": 725.25,
         